In [21]:
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
#from statsmodels.api import datasets
from sklearn import datasets ## Get dataset from sklearn
import sklearn.model_selection as ms
import sklearn.metrics as sklm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.random as nr
import itertools
from sklearn import cross_validation
import math

%matplotlib inline

In [22]:
bikesData = pd.read_csv('BikesProcessed.csv')
Labels = np.array(bikesData['BikeBuyer'])
bikesData.drop(bikesData.columns[0], axis=1, inplace=True)
bikesData.drop(['BikeBuyer'], axis=1, inplace=True)
Features = np.array(bikesData)
print(Features.shape)
bikesData.head()

(16404, 12)


,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,AreaCode,Sex,Married,JobType,EducationType,Country,Income,AgeBracket,ChildrenFlag
0,1,0,0,500,1,1,0,0,1,3,0,1
1,0,1,3,500,1,0,0,0,1,3,1,1
2,1,1,3,500,1,1,0,0,1,2,1,1
3,0,1,0,500,0,0,0,0,1,2,0,0
4,1,4,5,500,0,0,0,0,1,2,0,1


In [23]:
feature_combs = []

for i in range(1, 13):
    feature_combs.extend(list(itertools.combinations(list(bikesData), i)))         
print(len(feature_combs))

4095


In [24]:
#Scale the data
scaler = preprocessing.StandardScaler().fit(Features)
Features = scaler.transform(Features)
titles = list(bikesData)
bikes_scaled = pd.DataFrame()
count = 0
Features = pd.DataFrame(Features)

for title in titles:
    bikes_scaled[title] = Features[count]
    count+=1

bikes_scaled.head()

/Users/grahamcooper/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,AreaCode,Sex,Married,JobType,EducationType,Country,Income,AgeBracket,ChildrenFlag
0,0.695306,-1.316615,-0.654523,-1.024189,0.989328,0.923656,-1.128274,-1.148595,-0.296932,1.246941,-1.011425,0.590773
1,-1.438215,-0.438729,1.326679,-1.024189,0.989328,-1.082654,-1.128274,-1.148595,-0.296932,1.246941,0.288843,0.590773
2,0.695306,-0.438729,1.326679,-1.024189,0.989328,0.923656,-1.128274,-1.148595,-0.296932,0.398228,0.288843,0.590773
3,-1.438215,-0.438729,-0.654523,-1.024189,-1.010787,-1.082654,-1.128274,-1.148595,-0.296932,0.398228,-1.011425,-1.692697
4,0.695306,2.194929,2.647481,-1.024189,-1.010787,-1.082654,-1.128274,-1.148595,-0.296932,0.398228,-1.011425,0.590773


In [25]:
cv = ms.ShuffleSplit(n_splits=8, test_size=5000, random_state=0)
Labels = Labels.reshape(Labels.shape[0],)
scoring = ['accuracy']#,'precision_macro', 'recall_macro', 'f1_macro', 'roc_auc']

In [26]:
size = []
accuracy = []
#f1 = []
#precision = []
#recall = []
#auc = []
nr.seed(1115)
for comb in feature_combs:
    comb_features = np.array(bikes_scaled[np.array(comb)])
    nb_mod = GaussianNB()
    scores = ms.cross_validate(nb_mod, comb_features, Labels, scoring=scoring,
                        cv=cv, return_train_score=False)

    accuracy.append(np.mean(scores['test_accuracy']))
    #f1.append(np.mean(scores['test_f1_macro']))
    #precision.append(np.mean(scores['test_precision_macro']))
    #recall.append(np.mean(scores['test_recall_macro']))
    #auc.append(np.mean(scores['test_roc_auc']))
    size.append(len(comb))

In [27]:
midx = pd.DataFrame()
midx['size'] = size
midx['accuracy'] = accuracy
#midx['precision'] = precision
#midx['recall'] = recall
#midx['f1'] = f1
#midx['AUC'] = auc
midx['combs'] = feature_combs
midx.to_csv('CV_NaiveBayes.csv')